# Two-particle self consistency (TPSC)

$$ \chi_{sp}(k, i\omega_n) \equiv \frac{\chi_0(k, i\omega_n)}{1 - \frac{U_{sp}}{2} \chi_0(k, i\omega_n)} $$

$$ \chi_{ch}(k, i\omega_n) \equiv \frac{\chi_0(k, i\omega_n)}{1 + \frac{U_{ch}}{2} \chi_0(k, i\omega_n)} $$

$$ \mathrm{Tr} [ \chi ] \equiv \frac{1}{\beta N_k} \sum_{n, k} \chi(k, i\omega_n) $$

In [ ]:
from pytriqs.gf import Gf, inverse
from pytriqs.archive import HDFArchive
import numpy as np
with HDFArchive("tpsc.h5", 'r') as R:
    chi0_kw = R['chi0_kw']

In [ ]:
from scipy.optimize import fsolve

def chi_rpa(chi0_kw, U):
    chi = chi0_kw.copy()
    chi = chi * inverse(1 - float(U) * chi)
    return chi

# Should we have some kind of trace in TRIQS?
def trace_chi_kw(chi_kw):
    kmesh, wmesh = chi_kw.mesh.components
    # tail correction FIXME!!!
    # We neglect the tail. It is justified at those temperatures
    trace = chi_kw.data.sum() / len(kmesh) / wmesh.beta
    assert(np.abs(trace.imag) < 1e-10)
    return trace.real

def Usp_root_problem(Usp, chi0, n, U):
    tr_chi_sp = trace_chi_kw(chi_rpa(chi0, U=Usp[0]))
    diff = 2*tr_chi_sp + 0.5 * Usp/U * n**2 - n
    return diff

def Uch_root_problem(Uch, chi0, n, U, docc):
    tr_chi_ch = trace_chi_kw(chi_rpa(chi0, U=-Uch[0]))
    diff = 2*tr_chi_ch - 2 * docc - n + n**2
    return diff

def solve_Usp_and_Uch(chi0, U, n, Usp0=0.1, Uch0=0.1):
    Usp = fsolve(Usp_root_problem, Usp0, args=(chi0, n, U), xtol=1e-2)[0]
    docc = 0.25 * Usp / U * n**2
    Uch = fsolve(Uch_root_problem, Uch0, args=(chi0, n, U, docc), xtol=1e-2)[0]
    return Usp, Uch, docc

In [ ]:
n = 1.0
Usp, Uch = 0.3, 0.3

U_vec = np.concatenate((np.arange(0.3, 1., 0.2), np.arange(1., 6., 1.)))    
Usp_vec, Uch_vec, docc_vec = [np.zeros_like(U_vec) for x in xrange(3)]

print ''.join('| %-11s' % s for s in ['n', 'U', 'Usp', 'Uch', 'docc']), '|'
print '-'*67

for idx, U in enumerate(U_vec):
    Usp, Uch, docc = solve_Usp_and_Uch(chi0_kw, U, n, Usp0=Usp, Uch0=Uch)
    Usp_vec[idx], Uch_vec[idx], docc_vec[idx] = Usp, Uch, docc
    print ''.join('| %4.4E ' % x for x in [n, U, Usp, Uch, docc]), '|'

## Reproduce paper figure

We want to reproduce the following figure from the
__[paper](https://jp1.journaldephysique.org/articles/jp1/abs/1997/11/jp1v7p1309/jp1v7p1309.html)__

<img src="./img/Fig2.png" alt="Drawing" style="width: 250px;"/>

by calling the function tpsc for a grid of values of the bare U.


In [ ]:
%matplotlib inline
from matplotlib.pylab import plt
from matplotlib.image import imread


plt.figure(figsize=(8,8))

# Some manual adjustements here. May change from one
# machine to another
im = imread("img/Fig2.png")
plt.imshow(im, extent=(-0.45, 5.35, -3, 20.5), aspect='auto')
plt.plot([0,5,5,0,0],[0,0,20,20,0],'-r')

plt.plot(U_vec, Usp_vec, 'o-', label=r'$U_{sp}$', alpha=1, lw=2)
plt.plot(U_vec, Uch_vec, 'o-', label=r'$U_{ch}$', alpha=1, lw=2)

plt.axis('off')

### Note: weakness of RPA

Given the above sum rules, note that in TPSC the following sum-rule, a consequence of the Pauli principle, is satisfied:

\begin{equation}
\frac{T}{N}\sum_{\mathbf{q},iq_n} \left (\frac{\chi_0(\mathbf{q},iq_n)}{1-\frac{U_{sp}}{2}\chi_0(\mathbf{q},iq_n)}+\frac{\chi_0(\mathbf{q},iq_n)}{1+\frac{U_{ch}}{2}\chi_0(\mathbf{q},iq_n)}\right)=2n-n^2.
\end{equation}

Note that the right-hand side is independent of interactions.

    Compute the same quantity, but in the RPA approximation, 
    and make a plot of how much RPA violates the Pauli principle.

In [ ]:
## TODO !!